In [ ]:
"""
Created on Wed Jun 09 14:36 2021

Prepare proof of concept with a very simple DNN to parameterise the sub-shelf melt - more advanced than 1st try

Author: @claraburgard

"""

In [ ]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm
import glob
import matplotlib as mpl
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from basal_melt_neural_networks.constants import *
import basal_melt_neural_networks.diagnostic_functions as diag
import basal_melt_neural_networks.data_formatting as dfmt

import cartopy
import cartopy.crs as ccrs

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
nemo_run = 'OPM006'
inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
inputpath_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
inputpath_boxes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_melt = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
outputpath_nn = '/bettik/burgardc/SCRIPTS/basal_melt_neural_networks/data/interim/'

FOR EACH POINT:
- T and S profiles at the front (decompose z dimension into single things)
- Distance to front
- Distance to the grounding line
- Local slope ice draft
- Local slope bedrock
- Ice draft depth
- Bathymetry
- Ice draft concentration
- Horizontal coordinates (lon, lat)
- Mean bathymetry at entry (to add in future)
- Max bathymetry (to add in future)
- Target: melt m ice per yr

In [ ]:
# dIF, dGL, longitude, latitude
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

In [ ]:
# T and S profiles
file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018.nc')
file_TS = file_TS_orig.sel(Nisf=file_isf.Nisf)
file_TS_dom = file_TS.sel(profile_domain=50)

In [ ]:
plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics.nc')
# Local slope
local_ice_slope = plume_charac['alpha'].sel(option='appenB').drop('option')

In [ ]:
map_lim = [-3000000,3000000]
file_mask_orig = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')
file_mask_orig_cut = dfmt.cut_domain_stereo(file_mask_orig, map_lim, map_lim)
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = dfmt.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = dfmt.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:
# bathymetry, ice draft, concentration
file_bed_orig = file_mask_orig_cut['bathy_metry']
file_draft = file_other_cut['corrected_isfdraft'] 
file_isf_conc = file_conc_cut['isfdraft_conc']

In [ ]:
file_bedrock_slope = xr.open_dataset(inputpath_mask+'nemo_5km_bedrock_slope.nc')
local_bedrock_slope = file_bedrock_slope['bedrock_slope']

In [ ]:
NEMO_melt_rates_2D = xr.open_mfdataset(outputpath_melt+'melt_rates_2D_NEMO.nc')
melt_rate = NEMO_melt_rates_2D['melt_m_ice_per_y']

Collect all 2D data in one dataset

In [ ]:
# left out 'longitude', 'latitude'
geometry_2D = file_isf[['dGL', 'dIF']].merge(local_ice_slope).merge(local_bedrock_slope).merge(file_draft).merge(file_bed_orig)

SUBSAMPLE DATA

Select one ice shelf

In [ ]:
kisf_of_int = 66

In [ ]:
geometry_2D_isf = geometry_2D.where(file_isf['ISF_mask'] == kisf_of_int, drop=True)
melt_rate_isf = melt_rate.where(file_isf['ISF_mask'] == kisf_of_int, drop=True).load()
TS_isf = file_TS_dom.sel(Nisf=kisf_of_int)
max_front_depth = file_isf['front_bot_depth_max'].sel(Nisf=kisf_of_int)

PREPARE DATAFRAME WITH ALL DATA

In [ ]:
merged_yy_df = None
for kk, yyyy in enumerate(tqdm(range(1980,2010))):
    clean_df_yy, T_list, S_list = dfmt.prepare_input_df_1_year(TS_isf, melt_rate_isf, yyyy, max_front_depth, geometry_2D_isf)
    if kk > 0:
        merged_yy_df = merged_yy_df.append(clean_df_yy, ignore_index = True)
    else:
        merged_yy_df = clean_df_yy.copy()

In [ ]:
merged_yy_df

In [ ]:
clean_df = merged_yy_df.drop(['time'], axis=1)

In [ ]:
clean_df.to_csv(outputpath_nn + 'input_data_1980-2010_isf'+str(kisf_of_int).zfill(3)+'.csv')

DIVIDE INTO TRAIN AND TEST DATASET

In [ ]:
#all_files = glob.glob(outputpath_nn + 'input_data_1980-2010_isf*.csv')
#clean_df = pd.concat((pd.read_csv(f) for f in all_files)).drop('Unnamed: 0', 1)
#clean_df

In [ ]:
# if only one ice shelf
clean_df = pd.read_csv(outputpath_nn + 'input_data_1980-2010_isf'+str(kisf_of_int).zfill(3)+'.csv', index_col=0)

In [ ]:
clean_df

In [ ]:
T_list_drop = [ ]
S_list_drop = [ ]
for ii in range(60,85):
    T_list_drop.append('T_'+str(ii).zfill(3))
    S_list_drop.append('S_'+str(ii).zfill(3))

In [ ]:
T_list = [ ]
S_list = [ ]
for ii in range(60):
    T_list.append('T_'+str(ii).zfill(3))
    S_list.append('S_'+str(ii).zfill(3))


In [ ]:
clean_df = clean_df.drop(T_list_drop, 1)
clean_df = clean_df.drop(S_list_drop, 1)

In [ ]:
clean_df = clean_df.drop('melt_cavity', 1)

In [ ]:
data_train = clean_df.sample(frac=0.7, axis=0) 
data_test  = clean_df.drop(data_train.index)

In [ ]:
y_train = data_train['melt_m_ice_per_y']
x_train = data_train.drop(['melt_m_ice_per_y'], axis=1)

y_test = data_test['melt_m_ice_per_y']
x_test = data_test.drop(['melt_m_ice_per_y'], axis=1)

print('Original data shape was : ',clean_df.shape)
print('x_train : ',x_train.shape, 'y_train : ',y_train.shape)
print('x_test  : ',x_test.shape,  'y_test  : ',y_test.shape)

### 3.2 - Data normalization
**Note :** 
 - All input data must be normalized, train and test.  
 - To do this we will **subtract the mean** and **divide by the standard deviation**.  
 - But test data should not be used in any way, even for normalization.  
 - The mean and the standard deviation will therefore only be calculated with the train data.

In [ ]:
#display(x_train.describe().style.format("{0:.2f}").set_caption("Before normalization :"))

x_train_norm = x_train.copy()
x_test_norm = x_test.copy()

for ccol in ['dGL','dIF','alpha','bedrock_slope','corrected_isfdraft','bathy_metry','longitude','latitude']:
    mean = x_train[ccol].mean()
    std  = x_train[ccol].std()
    x_train_norm[ccol] = (x_train[ccol] - mean) / std
    x_test_norm[ccol]  = (x_test[ccol]  - mean) / std

mean_T = x_train[T_list].mean().mean()
std_T = x_train[T_list].mean().std()
mean_S = x_train[S_list].mean().mean()
std_S = x_train[S_list].mean().std()


for ccol in [T_list]:
    x_train_norm[ccol] = (x_train[ccol] - mean_T) / std_T
    x_test_norm[ccol] = (x_test[ccol] - mean_T) / std_T

for ccol in [S_list]:
    x_train_norm[ccol] = (x_train[ccol] - mean_S) / std_S
    x_test_norm[ccol] = (x_test[ccol] - mean_S) / std_S

#display(x_train.describe().style.format("{0:.2f}").set_caption("After normalization :"))
#display(x_train.head(5).style.format("{0:.2f}").set_caption("Few lines of the dataset :"))

x_train_arr, y_train_arr = np.array(x_train_norm), np.array(y_train)
x_test_arr,  y_test_arr  = np.array(x_test_norm),  np.array(y_test)


In [ ]:
x_train_arr

## Step 4 - Build a model
About informations about : 
 - [Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)
 - [Activation](https://www.tensorflow.org/api_docs/python/tf/keras/activations)
 - [Loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses)
 - [Metrics](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

In [ ]:
def get_model_v1(shape):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape, name="InputLayer"))
    model.add(keras.layers.Dense(32, activation='relu', name='Dense_n1'))
    model.add(keras.layers.Dense(64, activation='relu', name='Dense_n2'))
    model.add(keras.layers.Dense(32, activation='relu', name='Dense_n3'))
    model.add(keras.layers.Dense(1, name='Output'))
    
    model.compile(optimizer = 'adam',
                  loss      = 'mse',
                  metrics   = ['mae', 'mse'] )
    return model

def get_model_v2(shape):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape, name="InputLayer"))
    model.add(keras.layers.Dense(1, name='Output'))
    
    model.compile(optimizer = 'adam',
                  loss      = 'mse',
                  metrics   = ['mae', 'mse'] )
    return model

#def get_model_v1(shape):
#    nodes = 256
#    #activ = 'sigmoid'
#   activ = 'relu'   # standard
#    #activ = 'tanh'
#    #activ = 'selu'
#    model = keras.models.Sequential()
#    model.add(keras.layers.Input(shape, name="InputLayer"))
#    model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n1'))
#    model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n2'))
#    model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n3'))
#    model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n4'))
#    model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n5'))
#    model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n6'))
#    model.add(keras.layers.Dense(1, name='Output'))  
#                                    # nbvect = number of elements of target vector
#
#    model.compile(optimizer = 'rmsprop',
#                  loss      = 'mse',                  # mse, mean quadratic error 
#              metrics   = ['mae', 'mse'] )        # mae =  mean absolute error
#    return model

## Step 5 - Train the model
### 5.1 - Get it

In [ ]:
input_size = len(x_train_arr[0])

In [ ]:
model=get_model_v1( (input_size,) )

model.summary()

### 5.2 - Train it

In [ ]:
history = model.fit(x_train_arr,
                    y_train_arr,
                    epochs          = 40,
                    batch_size      = 10,
                    verbose         = 1,
                    validation_data = (x_test_arr, y_test_arr))

## Step 6 - Evaluate
### 6.1 - Model evaluation
MAE =  Mean Absolute Error (between the labels and predictions)  
A mae equal to 3 represents an average error in prediction of $3k.

In [ ]:
score = model.evaluate(x_test_arr, y_test_arr, verbose=1)

print('x_test / loss      : {:5.4f}'.format(score[0]))
print('x_test / mae       : {:5.4f}'.format(score[1]))
print('x_test / mse       : {:5.4f}'.format(score[2]))

### 6.2 - Training history
What was the best result during our training ?

In [ ]:
df=pd.DataFrame(data=history.history)
display(df)

In [ ]:
print("min( val_mae ) : {:.4f}".format( min(history.history["val_mae"]) ) )

In [ ]:
diag.plot_history(history, plot={'MSE' :['mse', 'val_mse'],
                                'MAE' :['mae', 'val_mae'],
                                'LOSS':['loss','val_loss']})

In [ ]:
plt.close('all')

## Step 7 - Make a prediction
The data must be normalized with the parameters (mean, std) previously used.

In [ ]:
for kk, yyyy in enumerate(tqdm(range(2018,2019))):
    clean_df_yy_val, T_list, S_list = dfmt.prepare_input_df_1_year(TS_isf, melt_rate_isf, yyyy, max_front_depth, geometry_2D_isf)
    if kk > 0:
        merged_yy_df_val = merged_yy_df_val.append(clean_df_yy_val, ignore_index = True)
    else:
        merged_yy_df_val = clean_df_yy_val.copy()

In [ ]:
clean_df_val = merged_yy_df_val.drop(['time'], axis=1).reset_index().drop(['x'], axis=1).drop(['y'], axis=1)

In [ ]:
#clean_df_val = clean_df_val.drop(T_list_drop, 1)
#clean_df_val = clean_df_val.drop(S_list_drop, 1)
#clean_df_val = clean_df_val.drop('melt_cavity', 1)

In [ ]:
y_val = clean_df_val['melt_m_ice_per_y']
x_val = clean_df_val.drop(['melt_m_ice_per_y'], axis=1)

x_val_norm = x_val.copy()

for ccol in ['dGL','dIF','alpha','bedrock_slope','corrected_isfdraft','bathy_metry','longitude','latitude']:
    mean = x_train[ccol].mean()
    std  = x_train[ccol].std()
    x_val_norm[ccol] = (x_val[ccol] - mean) / std

mean_T = x_train[T_list].mean().mean()
std_T = x_train[T_list].mean().std()
mean_S = x_train[S_list].mean().mean()
std_S = x_train[S_list].mean().std()

for ccol in [T_list]:
    x_val_norm[ccol] = (x_val[ccol] - mean_T) / std_T

for ccol in [S_list]:
    x_val_norm[ccol] = (x_val[ccol] - mean_S) / std_S

#display(x_train.describe().style.format("{0:.2f}").set_caption("After normalization :"))
#display(x_train.head(5).style.format("{0:.2f}").set_caption("Few lines of the dataset :"))

x_val_arr, y_val_arr = np.array(x_val_norm), np.array(y_val)

#my_data=np.array(x_val_arr)#.reshape(1,13)

In [ ]:
x_val_arr.shape

In [ ]:
y_out = model.predict(x_val_arr)

In [ ]:
y_out

In [ ]:
y_out_pd_s = pd.Series(y_out[:,0],index=merged_yy_df_val.index,name='computed_melt') 
y_target_pd_s = pd.Series(y_val_arr,index=merged_yy_df_val.index,name='reference_melt') 

In [ ]:
y_out_xr = y_out_pd_s.to_xarray()
y_target_xr = y_target_pd_s.to_xarray()
y_to_compare = xr.merge([y_out_xr.T, y_target_xr.T])

In [ ]:
xx = range(0,80)
plt.figure()
plt.scatter(y_to_compare['computed_melt'].values.flatten(),y_to_compare['reference_melt'].values.flatten(), s=10, edgecolors='None',alpha=0.2)
plt.plot(xx,xx,'k')

In [ ]:
computed_melt = y_to_compare['computed_melt']
ref_melt = y_to_compare['reference_melt']

In [ ]:
min_m = min(computed_melt.min(), ref_melt.min())
max_m = max(computed_melt.max(), ref_melt.max())
lim = max(abs(min_m),abs(max_m))

if min_m < 0:
    cmap = mpl.cm.coolwarm
    minlim = -lim
else:
    cmap = mpl.cm.viridis
    minlim = 0

f = plt.figure(figsize=(15, 5))

ax1 = plt.subplot(1, 3, 1)
computed_melt.plot(ax=ax1, vmin=minlim,vmax=lim, cmap=cmap)
ax1.set_title('Neural Network [m ice/y]')

ax2 = plt.subplot(1, 3, 2, sharex = ax1, sharey = ax1)
ref_melt.plot(ax=ax2, vmin=minlim,vmax=lim, cmap=cmap)
ax2.set_title('Reference [m ice/y]')

ax3 = plt.subplot(1, 3, 3, sharex = ax1, sharey = ax1)
(computed_melt - ref_melt).plot(ax=ax3)
ax3.set_xticklabels('')
ax3.set_yticklabels('')
ax3.set_title('NN - Ref [m ice/y]')

f.tight_layout()

In [ ]:
y_to_compare.to_netcdf(outputpath_nn+'prediction_linreg_2018_withoutlatlon.nc')

In [ ]:
y_to_compare_nn = xr.open_dataset(outputpath_nn+'prediction_nn_2018.nc')

In [ ]:
(y_to_compare_nn - y_to_compare)['computed_melt']

In [ ]:
rmse = np.sqrt(np.mean((y_out - y_val_arr)**2))

In [ ]:
rmse

In [ ]:
predictions = model.predict( my_data )
print("Prediction : {:.2f} m ice per y".format(predictions[0][0]))
print("Reality    : {:.2f} m ice per y".format(y_val_arr[0]))

COMPARE WITH SIMPLE

In [ ]:
sns.set_context('paper')

In [ ]:
simple_param = xr.open_dataset(outputpath_melt+'melt_rates_2D_quadratic_mixed_locslope_tuned_correctedTS.nc')
melt_simple = simple_param['melt_m_ice_per_y'].sel(profile_domain=50,time=2018).where(file_isf['ISF_mask']==66, drop=True)

In [ ]:
y_to_compare_nn = xr.open_dataset(outputpath_nn+'prediction_linreg_2018_withoutlatlon.nc')
computed_melt = y_to_compare_nn['computed_melt']
ref_melt = y_to_compare_nn['reference_melt']

In [ ]:
plot_path = '/bettik/burgardc/PLOTS/generic_plots/'
min_m = min(computed_melt.min(), ref_melt.min())
max_m = max(computed_melt.max(), ref_melt.max())
lim = max(abs(min_m),abs(max_m))

if min_m < 0:
    cmap = mpl.cm.coolwarm
    minlim = -lim
    cmap_diff = mpl.cm.BrBG_r
else:
    cmap = mpl.cm.viridis
    minlim = 0
    cmap_diff = mpl.cm.BuGn
    
f = plt.figure(figsize=(15, 5))

ax1 = plt.subplot(1, 3, 1)
ref_melt.plot(ax=ax1, vmin=minlim,vmax=lim, cmap=cmap)
ax1.set_title('Reference [m ice/y]')

ax2 = plt.subplot(1, 3, 2, sharex = ax1, sharey = ax1)
(y_to_compare_nn['computed_melt'] - ref_melt).plot(ax=ax2, vmin=minlim,vmax=lim, cmap=cmap_diff)
ax2.set_title('Neural Network param - Reference [m ice/y]')

ax3 = plt.subplot(1, 3, 3, sharex = ax1, sharey = ax1)
(melt_simple - ref_melt).plot(ax=ax3, vmin=minlim,vmax=lim, cmap=cmap_diff)
ax3.set_xticklabels('')
ax3.set_yticklabels('')
ax3.set_title('Simple physical param - Reference [m ice/y]')

f.tight_layout()
f.savefig(plot_path+'comparison_proof_of_concept.png', dpi=300)

In [ ]:
plot_path = '/bettik/burgardc/PLOTS/generic_plots/'
min_m = min((computed_melt - ref_melt).min(), ref_melt.min())
max_m = max((computed_melt - ref_melt).max(), ref_melt.max())
lim = max(abs(min_m),abs(max_m))

if min_m < 0:
    cmap = mpl.cm.coolwarm
    minlim = -lim
    cmap_diff = mpl.cm.BrBG_r
else:
    cmap = mpl.cm.viridis
    minlim = 0
    cmap_diff = mpl.cm.copper

f = plt.figure()
f.set_size_inches(8.24*1.3, 8.24/3)

ax1 = plt.subplot(1, 3, 1)
ref_melt.plot(ax=ax1, vmin=minlim,vmax=lim, cmap=cmap)
ax1.set_title('Reference [m ice/y]')

ax2 = plt.subplot(1, 3, 2, sharex = ax1, sharey = ax1)
(y_to_compare_nn['computed_melt'] - ref_melt).plot(ax=ax2, vmin=minlim,vmax=lim, cmap=cmap_diff)
ax2.set_title('Neural Network param - Reference [m ice/y]')

ax3 = plt.subplot(1, 3, 3, sharex = ax1, sharey = ax1)
(melt_simple - ref_melt).plot(ax=ax3, vmin=minlim,vmax=lim, cmap=cmap_diff)
ax3.set_xticklabels('')
ax3.set_yticklabels('')
ax3.set_title('Simple physical param - Reference [m ice/y]')

f.tight_layout()
f.savefig(plot_path+'comparison_proof_of_concept_for_ANR_proposal_withoutlatlon.png', dpi=300)

In [ ]:
#ax = plt.axes(projection=ccrs.SouthPolarStereo(central_longitude=0,true_scale_latitude=-71))
#ax.coastlines(resolution='50m', linewidth=0.5)
#ax.pcolormesh(melt_simple.longitude,melt_simple.latitude,melt_simple,transform=ccrs.PlateCarree(),rasterized=True)
#ax.set_extent([-180, 180, -90, -60], crs=ccrs.PlateCarree())

======== TO KEEP FOR THE FUTURE =========

In [ ]:
# For each column - for normalization with min and max

normalized_clean_df = clean_df.copy()

for ccol in ['dGL','dIF','alpha','bedrock_slope','corrected_isfdraft','bathy_metry','longitude','latitude','melt_cavity','time']:
    max_ccol = clean_df[ccol].max()
    min_ccol = clean_df[ccol].min()
    normalized_clean_df[ccol] = (clean_df[ccol] - min_ccol)/(max_ccol - min_ccol)

max_T = clean_df[T_list].max().max()
min_T = clean_df[T_list].min().min()
max_S = clean_df[S_list].max().max()
min_S = clean_df[S_list].min().min()

for ccol in [T_list]:
    normalized_clean_df[ccol] = (clean_df[ccol] - min_T)/(max_T - min_T)

for ccol in [S_list]:
    normalized_clean_df[ccol] = (clean_df[ccol] - min_S)/(max_S - min_S)